# ETL_EDA
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from BPA 
The [dataset](https://transmission.bpa.gov/business/operations/Wind/baltwg.txt) is a continuously updated txt file in csv format. It contains the energy producation and load in the last 5 days. There will be some blank lines at the end for today. It can be retrieved simply by making `requests` without parameters. For more complicated API call, you may need to add query parameters.

In [113]:
import requests

url = "https://transmission.bpa.gov/business/operations/Wind/baltwg.txt"
req = requests.get(url, timeout=0.5)
req.raise_for_status()
text = req.text
print(text[:1000])
print('...')
print(text[-500:])

BPA Balancing Authority Load & Total Wind Generation
at 5-minute intervals, last 7 days
Dates: 30Nov2019 - 07Dec2019 (last updated 6Dec2019 13:30:34) Pacific Time
Based on 5-min MW readings from the BPA SCADA system for points 45583, 79687, 79682, 164377, 70681

This represents loads and resources in BPA's Balancing Authority (BA) including some that are not BPA's.
It does not include BPA loads served by transfer, scheduled out of region,
or scheduled to customers with their own BAs such as Seattle and Tacoma

BPA/Technical Operations (TOT-OpInfo@bpa.gov)

Date/Time       	Load	Wind	Hydro	Fossil/Biomass	Nuclear
11/30/2019 00:00	7143	35	6844	506	1168
11/30/2019 00:05	7146	35	6832	508	1170
11/30/2019 00:10	7122	31	6842	510	1167
11/30/2019 00:15	7138	29	6714	505	1170
11/30/2019 00:20	7097	27	6739	506	1168
11/30/2019 00:25	7086	26	6721	506	1169
11/30/2019 00:30	7119	26	6745	508	1167
11/30/2019 00:35	7117	23	6763	509	1167
11/30/2019 00:40	7128	19	6688	509	1169
11/30/201
...
2019 22:10					


In [106]:
import charts
import datetime
today = datetime.datetime.today() #2019-12-05#
today= today.strftime("%Y-%m-%d")
lastweek = datetime.datetime.today() - datetime.timedelta(days = 1)
lastweek = lastweek.strftime("%Y-%m-%d")
chart = charts.get_charts(lastweek, today,region='nl')
chart.head()

100%|██████████| 2/2 [00:02<00:00,  1.00s/it]

,Position,Track Name,Artist,Streams,URL,date
0,1,Dance Monkey,Tones and I,282587,https://open.spotify.com/track/1rgnBhdG2JDFTbY...,2019-12-05
1,2,Pa Olvidarte,Emma Heesters,230923,https://open.spotify.com/track/1RO8Q3w2Jkir8cv...,2019-12-05
2,3,Memories,Maroon 5,211134,https://open.spotify.com/track/2b8fOow8UzyDFAE...,2019-12-05
3,4,ROXANNE,Arizona Zervas,199803,https://open.spotify.com/track/696DnlkuDOXcMAn...,2019-12-05
4,5,everything i wanted,Billie Eilish,171956,https://open.spotify.com/track/3ZCTVFBt2Brf31R...,2019-12-05


## 2. Raw Data to Documents/Dicts
Using pandas, it is simple to parse a in-memory string. The first few lines of description need to be skipped. Datetime conversion is made and blank lines are dropped. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [114]:
import pandas
from io import StringIO

df = pandas.read_csv(StringIO(text), skiprows=11, delimiter='\t')
df.columns = df.columns.str.strip()             # remove space in columns name
df['Datetime'] = pandas.to_datetime(df['Date/Time'])
df.drop(columns=['Date/Time'], axis=1, inplace=True)
df.dropna(inplace=True)  
df.head()

,Load,Wind,Hydro,Fossil/Biomass,Nuclear,Datetime
0,7143.0,35.0,6844.0,506.0,1168.0,2019-11-30 00:00:00
1,7146.0,35.0,6832.0,508.0,1170.0,2019-11-30 00:05:00
2,7122.0,31.0,6842.0,510.0,1167.0,2019-11-30 00:10:00
3,7138.0,29.0,6714.0,505.0,1170.0,2019-11-30 00:15:00
4,7097.0,27.0,6739.0,506.0,1168.0,2019-11-30 00:20:00


In [107]:
import pandas
import spotipy
import spotipy.util as util
import sys
# define authentication
username = 'zhiyanwang27'
scope = 'user-read-private'

CLIENT_ID = '1d35e049282a4f198bd09ff07e105784'
CLIENT_SECRET = '54850e7841cb47c5bc6c8970457dee89'

# playlist_id = '6watTtiqGlxPj2cxy6Sk8U'

token = util.prompt_for_user_token(username, 
                                   scope,
                                   client_id=CLIENT_ID,
                                   client_secret=CLIENT_SECRET,
                                   redirect_uri='http://localhost/8000/')

sp = spotipy.Spotify(auth=token)



In [108]:
df = chart
df['date'] = pandas.to_datetime(df['date'])
df['ID'] = 'default value'
df['genre'] = 'default value'
df['follwers'] = 'default value'
for i in range(df.shape[0]):
    track_id = df.iloc[i,4][31:]
    df.iloc[i,6] = track_id
    artist_id = sp.track(track_id)['artists'][0]['id']
    if len(sp.artist(artist_id)['genres']) > 0:
        gr = sp.artist(artist_id)['genres'][0]
    else:
        gr = 'None'
    fl = sp.artist(artist_id)['followers']['total']
    df.iloc[i,7] = gr
    df.iloc[i,8] = fl
df

,Position,Track Name,Artist,Streams,URL,date,ID,genre,follwers
0,1,Dance Monkey,Tones and I,282587,https://open.spotify.com/track/1rgnBhdG2JDFTbY...,2019-12-05,1rgnBhdG2JDFTbYkYRZAku,australian pop,289858
1,2,Pa Olvidarte,Emma Heesters,230923,https://open.spotify.com/track/1RO8Q3w2Jkir8cv...,2019-12-05,1RO8Q3w2Jkir8cvvGEApI0,channel pop,46336
2,3,Memories,Maroon 5,211134,https://open.spotify.com/track/2b8fOow8UzyDFAE...,2019-12-05,2b8fOow8UzyDFAE27YhOZM,pop,21061494
3,4,ROXANNE,Arizona Zervas,199803,https://open.spotify.com/track/696DnlkuDOXcMAn...,2019-12-05,696DnlkuDOXcMAnKlTgXXK,pop rap,118012
4,5,everything i wanted,Billie Eilish,171956,https://open.spotify.com/track/3ZCTVFBt2Brf31R...,2019-12-05,3ZCTVFBt2Brf31RLEnCkWJ,electropop,16600385
5,6,Circles,Post Malone,171042,https://open.spotify.com/track/21jGcNKet2qwijl...,2019-12-05,21jGcNKet2qwijlDFuPiPb,dfw rap,19904838
6,7,Reünie,Snelle,163510,https://open.spotify.com/track/5S7YAM3lcfPNBST...,2019-12-05,5S7YAM3lcfPNBSTVyciwCu,dutch hip hop,126463
7,8,Someone You Loved,Lewis Capaldi,161499,https://open.spotify.com/track/7qEHsqek33rTcFN...,2019-12-05,7qEHsqek33rTcFNT9PFqLf,pop,1711626
8,9,Don't Start Now,Dua Lipa,156678,https://open.spotify.com/track/6WrI0LAC5M1Rw2M...,2019-12-05,6WrI0LAC5M1Rw2MnX2ZvEg,dance pop,14379060
9,10,Lippenstift,Marco Borsato,143881,https://open.spotify.com/track/3D9MKbMaHCZjSEc...,2019-12-05,3D9MKbMaHCZjSEcCWf4wDm,dutch cabaret,441126


## 3. Upsert MongoDB
If we fetch the data frequently, there are lots of duplicate data entry between each run. The de-duplication happens at insertion. The MongoDB API to use is `collection.replace_one(filter=..., replacement=..., upsert=True)`. The statement matches a document in MongoDB with `filter`, replaces it with `replacement` if the document exists or inserts `replacement` into the database if `filter` matches nothing. 

In [116]:
import pymongo

client = pymongo.MongoClient()

In [117]:
db = client.get_database("energy")
collection = db.get_collection("energy")
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter={'Datetime': record['Datetime']},    # locate the document if exists
        replacement=record,                         # latest document
        upsert=True)                                # update if exists, insert if not
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

In [111]:
db = client.get_database("spotify")
collection = db.get_collection("spotify")
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter={'date': record['date']},    # locate the document if exists
        replacement=record,                         # latest document
        upsert=True)                                # update if exists, insert if not
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

In [112]:
df.to_dict

<bound method DataFrame.to_dict of      Position                                         Track Name  \
0           1                                       Dance Monkey   
1           2                                       Pa Olvidarte   
2           3                                           Memories   
3           4                                            ROXANNE   
4           5                                everything i wanted   
5           6                                            Circles   
6           7                                             Reünie   
7           8                                  Someone You Loved   
8           9                                    Don't Start Now   
9          10                                        Lippenstift   
10         11                                            Ride It   
11         12                                      Hoe Het Danst   
12         13                    All I Want for Christmas Is You   
13         14